# Boolean Logic and Regulation Motifs in Genetics
- http://be150.caltech.edu/2020/content/lessons/05_ffls.html
- https://www.nature.com/articles/nrg2102

In [2]:
# import biocircuits
import numpy as np
import scipy.integrate

import bokeh
import bokeh.io
import bokeh.plotting
import bokeh.layouts
import colorcet

# import panel as pn

bokeh.io.output_notebook()
# pn.extension()

colors = colorcet.b_glasbey_category10

Loading BokehJS ...

import numpy as np
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

output_notebook()

x = np.linspace(0,4*np.pi, 100)
y = np.sin(x)
plot = figure(width=400, height=400)
plot.line(x,y)
show(plot)

In [3]:
def contourf(x, y, z, title=None, palette="Viridis256"):
    """Make a filled contour plot given x, y, z data given in 2D arrays."""
    p = bokeh.plotting.figure(
        frame_height=200,
        frame_width=200,
        x_range=(x.min(), x.max()),
        y_range=(y.min(), y.max()),
        x_axis_label="x",
        y_axis_label="y",
        title=title,
    )
    p.image(
        image=[z],
        x=x.min(),
        y=y.min(),
        dw=x.max() - x.min(),
        dh=x.max() - x.min(),
        palette=palette,
        alpha=0.8,
    )

    return p


# Get x and y values for plotting
x = np.linspace(0, 2, 200)
y = np.linspace(0, 2, 200)
xx, yy = np.meshgrid(x, y)

# Parameters (steep Hill functions)
nx = 20
ny = 20

# Regulation functions
def aa_and(x, y, nx, ny):
    return x ** nx * y ** ny / (1 + x ** nx * y ** ny)


def aa_or(x, y, nx, ny):
    return (x ** nx + y ** ny) / (1 + x ** nx + y ** ny)


# Generate plots
plots = [
    contourf(xx, yy, aa_and(xx, yy, nx, ny), title="two activators, AND logic"),
    contourf(xx, yy, aa_or(xx, yy, nx, ny), title="two activators, OR logic"),
]

# bokeh.io.show(bokeh.layouts.row(plots), notebook_handle=True)
bokeh.io.show(bokeh.layouts.row(plots), notebook_handle=True)

In [15]:
def solve_ffl(
    beta, gamma, kappa, n_xy, n_xz, n_yz, ffl, logic, t, t_step_down, x_0
):
    """Solve an FFL. The dynamics are given by
    `rhs`, the output of `ffl_rhs()`.
    """
    if t[0] != 0:
        raise RuntimeError("time must start at zero.")

    rhs = ffl_rhs(beta, gamma, kappa, n_xy, n_xz, n_yz, ffl, logic)

    # Integrate if we do not step down
    if t[-1] < t_step_down:
        return scipy.integrate.odeint(rhs, np.zeros(2), t, args=(x_0,))

    # Integrate up to step down
    t_during_step = np.concatenate((t[t < t_step_down], (t_step_down,)))
    yz_during_step = scipy.integrate.odeint(
        rhs, np.zeros(2), t_during_step, args=(x_0,)
    )

    # Integrate after step
    t_after_step = np.concatenate(((t_step_down,), t[t > t_step_down]))
    yz_after_step = scipy.integrate.odeint(
        rhs, yz_during_step[-1, :], t_after_step, args=(0,)
    )

    # Concatenate solutions
    if t_step_down in t:
        return np.vstack((yz_during_step[:-1, :], yz_after_step))
    else:
        return np.vstack((yz_during_step[:-1, :], yz_after_step[1:, :]))

def ffl_rhs(beta, gamma, kappa, n_xy, n_xz, n_yz, ffl, logic):
    """Return a function with call signature fun(yz, x) that computes
    the right-hand side of the dynamical system for an FFL. Here,
    `yz` is a length two array containing concentrations of Y and Z.
    """
    if ffl[:2].lower() in ("c1", "c3", "i1", "i3"):
        fy = lambda x: biocircuits.reg.act_hill(x, n_xy)
    else:
        fy = lambda x: biocircuits.reg.rep_hill(x, n_xy)

    if ffl[:2].lower() in ("c1", "i4"):
        if logic.lower() == "and":
            fz = lambda x, y: biocircuits.reg.aa_and(x, y, n_xz, n_yz)
        else:
            fz = lambda x, y: biocircuits.reg.aa_or(x, y, n_xz, n_yz)
    elif ffl[:2].lower() in ("c4", "i1"):
        if logic.lower() == "and":
            fz = lambda x, y: biocircuits.reg.ar_and(x, y, n_xz, n_yz)
        else:
            fz = lambda x, y: biocircuits.reg.ar_or(x, y, n_xz, n_yz)
    elif ffl[:2].lower() in ("c2", "i3"):
        if logic.lower() == "and":
            fz = lambda x, y: biocircuits.reg.ar_and(y, x, n_yz, n_xz)
        else:
            fz = lambda x, y: biocircuits.reg.ar_or(y, x, n_yz, n_xz)
    else:
        if logic.lower() == "and":
            fz = lambda x, y: biocircuits.reg.rr_and(x, y, n_xz, n_yz)
        else:
            fz = lambda x, y: biocircuits.reg.rr_or(x, y, n_xz, n_yz)

    def rhs(yz, t, x):
        y, z = yz
        dy_dt = beta * fy(kappa * x) - y
        dz_dt = gamma * (fz(x, y) - z)

        return np.array([dy_dt, dz_dt])

    return rhs

In [16]:
param_sliders_kwargs = dict(start=0.1, end=10, step=0.1, value=1)
hill_coeff_kwags = dict(start=0.1, end=10, step=0.1, value=1)

beta_slider = pn.widgets.FloatSlider(name="β", **param_sliders_kwargs)
gamma_slider = pn.widgets.FloatSlider(name="γ", **param_sliders_kwargs)
kappa_slider = pn.widgets.FloatSlider(name="κ", **param_sliders_kwargs)
n_xy_slider = pn.widgets.FloatSlider(name="nxy", **hill_coeff_kwags)
n_xz_slider = pn.widgets.FloatSlider(name="nxz", **hill_coeff_kwags)
n_yz_slider = pn.widgets.FloatSlider(name="nyz", **hill_coeff_kwags)
ffl_selector = pn.widgets.Select(
    name="Circuit",
    options=[
        f"{x}-FFL" for x in ["C1", "C2", "C3", "C4", "I1", "I2", "I3", "I4"]
    ],
    value="C1-FFL",
)
logic_selector = pn.widgets.RadioBoxGroup(
    name="Logic", options=["AND", "OR"], value="AND", inline=True
)
t_step_down_slider = pn.widgets.FloatSlider(
    name="step down time", start=0.1, end=21, step=0.1, value=10
)
x_0_slider = pn.widgets.FloatSlider(
    name="x₀", start=0.1, end=10, step=0.1, value=1
)
normalize_checkbox = pn.widgets.Checkbox(name="Normalize", value=False)





In [17]:
@pn.depends(
    beta_slider.param.value,
    gamma_slider.param.value,
    kappa_slider.param.value,
    n_xy_slider.param.value,
    n_xz_slider.param.value,
    n_yz_slider.param.value,
    ffl_selector.param.value,
    logic_selector.param.value,
    t_step_down=t_step_down_slider.param.value,
    x_0=x_0_slider.param.value,
    normalized=normalize_checkbox.param.value,
)
def plot_ffl(
    beta=1,
    gamma=1,
    kappa=1,
    n_xy=1,
    n_xz=1,
    n_yz=1,
    ffl="c1",
    logic="and",
    t=np.linspace(0, 20, 200),
    t_step_down=10,
    x_0=1,
    normalized=False,
):
    yz = solve_ffl(
        beta, gamma, kappa, n_xy, n_xz, n_yz, ffl, logic, t, t_step_down, x_0
    )
    y, z = yz.transpose()

    # Generate x-values
    if t[-1] > t_step_down:
        t_x = np.array(
            [-t_step_down / 10, 0, 0, t_step_down, t_step_down, t[-1]]
        )
        x = np.array([0, 0, x_0, x_0, 0, 0])
    else:
        t_x = np.array([-t[-1] / 10, 0, 0, t[-1]])
        x = np.array([0, 0, x_0, x_0])

    # Add left part of y and z-values
    t = np.concatenate(((t_x[0],), t))
    y = np.concatenate(((0,), y))
    z = np.concatenate(((0,), z))

    # Set up figure
    p = bokeh.plotting.figure(
        frame_height=175,
        frame_width=550,
        x_axis_label="dimensionless time",
        y_axis_label=f"{'norm. ' if normalized else ''}dimensionless conc.",
        x_range=[t.min(), t.max()],
    )

    if normalized:
        p.line(
            t_x, x / x.max(), line_width=2, color=colors[0], legend_label="x"
        )
        p.line(
            t, y / y.max(), line_width=2, color=colors[1], legend_label="y",
        )
        p.line(
            t, z / z.max(), line_width=2, color=colors[2], legend_label="z",
        )
    else:
        p.line(t_x, x, line_width=2, color=colors[0], legend_label="x")
        p.line(t, y, line_width=2, color=colors[1], legend_label="y")
        p.line(t, z, line_width=2, color=colors[2], legend_label="z")

    p.legend.click_policy = "hide"

    return p

In [18]:
sliders = pn.Row(
    pn.Spacer(width=30),
    pn.Column(beta_slider, gamma_slider, kappa_slider, width=150),
    pn.Spacer(width=10),
    pn.Column(n_xy_slider, n_xz_slider, n_yz_slider, width=150),
    pn.Spacer(width=10),
    pn.Column(t_step_down_slider, x_0_slider, width=150),
)

selectors = pn.Column(
    ffl_selector, logic_selector, normalize_checkbox, width=150
)

# Final layout
pn.Column(
    plot_ffl,
    pn.Spacer(width=10),
    pn.Row(sliders, pn.Spacer(width=10), selectors),
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'c10bc141-6351-4d68-971a-bb5ffa748f7f': {'defs': …

### Example from Bokeh

In [19]:


p1 = figure(**opts)
r1 = p1.circle([1,2,3], [4,5,6], size=20)

p2 = figure(**opts)
r2 = p2.circle([1,2,3], [4,5,6], size=20)

# get a handle to update the shown cell with
t = show(row(p1, p2), notebook_handle=True)

